# Notebook 2: Selection and Indexing

Welcome to the second notebook! Now that you know how to load data, let's learn how to select and access specific parts of your DataFrames.

## What You'll Learn

- Selecting columns by name and by list
- Using `.loc[]` for label-based indexing
- Using `.iloc[]` for position-based indexing
- Boolean filtering (conditional selection)
- Setting and using index
- Common pitfalls and errors to avoid

## Prerequisites

- Completed Notebook 1 (Data Loading)
- Understand DataFrames and Series
- Sample datasets in `../data/` folder

In [ ]:
# Setup: imports and display options
import pandas as pd
import numpy as np

pd.options.display.max_columns = 50
pd.options.display.width = 120
pd.options.display.max_rows = 20

print("Setup complete!")
print(f"pandas version: {pd.__version__}")

## 1. Load Fresh Data

We'll start by loading the datasets fresh from the `../data/` folder:

**Note**: In a real workflow, you might load cleaned data from a previous notebook. See the commented section at the end for how to do that.

In [ ]:
# Load media contacts data
media_df = pd.read_csv('../data/media_contacts.csv')

# Standardize column names (lowercase, underscores)
media_df.columns = media_df.columns.str.replace(' ', '_').str.lower()

print(f"Loaded {len(media_df)} rows")
media_df.head()

In [ ]:
# Load demographic data
demo_df = pd.read_csv('../data/socio_demos.csv')

# Rename columns for consistency
demo_df = demo_df.rename(columns={
    'Person ID': 'person_id',
    'Number_of children': 'num_children',
    'People_in_Household': 'household_size'
})

print(f"Loaded {len(demo_df)} rows")
demo_df.head()

## 2. Selecting Columns

### Single Column Selection

Select one column - returns a **Series**:

In [ ]:
# Two ways to select a single column
tv_data = media_df['tv_total']
# OR
tv_data = media_df.tv_total  # Only works if column name has no spaces/special chars

print(type(tv_data))
print(f"\nFirst 5 values:")
print(tv_data.head())

### Multiple Column Selection

Select multiple columns using a **list** - returns a **DataFrame**:

In [ ]:
# Select specific columns
tv_online = media_df[['person_id', 'tv_total', 'online_total']]

print(f"Shape: {tv_online.shape}")
tv_online.head()

In [ ]:
# Select many columns at once
social_cols = ['person_id', 'tiktok', 'youtube_total', 'pinterest']
social_df = media_df[social_cols]

social_df.head()

### Column Reordering

You can reorder columns by selecting them in a different order:

In [ ]:
# Reorder: put person_id first, then purchase, then media columns
new_order = ['person_id', 'purchase', 'tv_total', 'online_total', 'print_total']
reordered = media_df[new_order]

reordered.head()

## 3. Row Selection with `.loc[]` and `.iloc[]`

### `.iloc[]` - Integer Location (Position-Based)

Use **integer positions** to select rows and columns:

In [ ]:
# Select first row (position 0)
first_row = media_df.iloc[0]
print("First row:")
print(first_row)

In [ ]:
# Select first 5 rows
first_five = media_df.iloc[0:5]
print(f"Shape: {first_five.shape}")
first_five

In [ ]:
# Select specific rows by position
rows_to_select = [0, 10, 20, 30]
selected = media_df.iloc[rows_to_select]
selected

In [ ]:
# Select rows AND columns by position
# Rows 0-5, columns 0-3
subset = media_df.iloc[0:5, 0:3]
print(f"Shape: {subset.shape}")
subset

### `.loc[]` - Label-Based Indexing

Uses **labels** (index and column names). With default integer index, it works similarly to `.iloc[]` but **includes** the end point:

In [ ]:
# Select rows 0 to 4 (INCLUDES 4, unlike iloc)
rows_loc = media_df.loc[0:4]
print(f"Shape: {rows_loc.shape}")  # 5 rows (0, 1, 2, 3, 4)
rows_loc

In [ ]:
# Select specific rows and columns by label
subset = media_df.loc[0:3, ['person_id', 'tv_total', 'online_total']]
subset

## 4. Boolean Filtering (Conditional Selection)

This is one of the most powerful features in pandas!

### Simple Conditions

In [ ]:
# Find rows where TV Total is greater than 50
high_tv = media_df[media_df['tv_total'] > 50]

print(f"Found {len(high_tv)} rows with TV Total > 50")
high_tv.head()

In [ ]:
# People who purchased (Purchase == 1)
purchasers = media_df[media_df['purchase'] == 1]

print(f"Purchasers: {len(purchasers)} out of {len(media_df)} ({len(purchasers)/len(media_df)*100:.1f}%)")
purchasers.head()

### Multiple Conditions

Combine conditions with `&` (and) or `|` (or). **Important**: Use parentheses around each condition!

In [ ]:
# High TV exposure AND purchased
high_tv_purchasers = media_df[(media_df['tv_total'] > 50) & (media_df['purchase'] == 1)]

print(f"High TV purchasers: {len(high_tv_purchasers)}")
high_tv_purchasers.head()

In [ ]:
# Either high TV OR high online
high_exposure = media_df[(media_df['tv_total'] > 50) | (media_df['online_total'] > 20)]

print(f"High exposure (TV or Online): {len(high_exposure)}")
high_exposure.head()

In [ ]:
# Complex condition: High TV AND (purchased OR high online)
complex_filter = media_df[
    (media_df['tv_total'] > 30) & 
    ((media_df['purchase'] == 1) | (media_df['online_total'] > 15))
]

print(f"Complex filter: {len(complex_filter)} rows")
complex_filter.head()

### `.isin()` for Multiple Values

Check if values are in a list:

In [ ]:
# Find specific person IDs
target_ids = [5326201, 256895, 7894945]
target_people = media_df[media_df['person_id'].isin(target_ids)]

target_people

### String Filtering (on demographic data)

In [ ]:
# Filter by gender
females = demo_df[demo_df['Gender'] == 'female']

print(f"Females: {len(females)} ({len(females)/len(demo_df)*100:.1f}%)")
females.head()

In [ ]:
# Filter by household size (string contains)
single_households = demo_df[demo_df['household_size'].str.contains('1')]

print(f"Single-person households: {len(single_households)}")
single_households.head()

## 5. Working with Index

### Setting an Index

Use a meaningful column as the row labels:

In [ ]:
# Set person_id as index (temporarily)
media_indexed = media_df.set_index('person_id')

print("Original index:")
print(media_df.index[:5])
print("\nNew index:")
print(media_indexed.index[:5])

media_indexed.head()

### Selecting by Index Labels

With a custom index, `.loc[]` uses those labels:

In [ ]:
# Select a specific person by ID
person_data = media_indexed.loc[5326201]

print("Data for person 5326201:")
print(person_data)

In [ ]:
# Select multiple people by ID
people_ids = [5326201, 256895, 7894945]
people_data = media_indexed.loc[people_ids]

people_data

### Resetting the Index

Convert index back to a regular column:

In [ ]:
# Reset index back to default integers
media_reset = media_indexed.reset_index()

media_reset.head()

## 6. Common Pitfalls and Errors

### Pitfall 1: Forgetting Parentheses in Boolean Conditions

In [ ]:
# ❌ WRONG - will cause error
try:
    wrong = media_df[media_df['tv_total'] > 50 & media_df['purchase'] == 1]
except Exception as e:
    print(f"Error: {e}")

# ✅ CORRECT - use parentheses
correct = media_df[(media_df['tv_total'] > 50) & (media_df['purchase'] == 1)]
print(f"\nCorrect result: {len(correct)} rows")

### Pitfall 2: Using `==` vs `.isin()` for Multiple Values

In [ ]:
# ❌ WRONG - checking if column equals a list (doesn't work as expected)
# wrong = media_df[media_df['person_id'] == [123, 456, 789]]

# ✅ CORRECT - use .isin() for multiple values
correct = media_df[media_df['person_id'].isin([5326201, 256895, 7894945])]
print(f"Correct way: {len(correct)} rows")

### Pitfall 3: `.loc[]` vs `.iloc[]` Confusion

In [ ]:
# .iloc uses POSITIONS (0-based, excludes end)
print("iloc[0:5] - Rows at positions 0, 1, 2, 3, 4:")
print(media_df.iloc[0:5].shape)  # 5 rows

# .loc uses LABELS (includes end)
print("\nloc[0:5] - Rows with labels 0, 1, 2, 3, 4, 5:")
print(media_df.loc[0:5].shape)  # 6 rows!

### Pitfall 4: Single vs Double Brackets

In [ ]:
# Single brackets - returns Series
series = media_df['tv_total']
print(f"Single brackets: {type(series)}")

# Double brackets - returns DataFrame
dataframe = media_df[['tv_total']]
print(f"Double brackets: {type(dataframe)}")

# This matters for chaining operations!
print(f"\nSeries shape: {series.shape}")
print(f"DataFrame shape: {dataframe.shape}")

## Summary

In this notebook, you learned:

✅ Select single columns with `df['column']` (returns Series)  
✅ Select multiple columns with `df[['col1', 'col2']]` (returns DataFrame)  
✅ Use `.iloc[]` for position-based indexing (0, 1, 2, ...)  
✅ Use `.loc[]` for label-based indexing (column/index names)  
✅ Filter with boolean conditions: `df[df['col'] > 10]`  
✅ Combine conditions with `&` (and) and `|` (or) - use parentheses!  
✅ Use `.isin()` for checking multiple values  
✅ Set custom index with `.set_index()` and reset with `.reset_index()`  
✅ Avoid common pitfalls with brackets, operators, and indexing

### Next Steps

In the next notebook (**03_cleaning_and_transformations.ipynb**), we'll learn:
- Data type conversions
- Parsing dates from various formats
- Handling missing values
- String operations and transformations
- Creating new columns

## 🎯 Practice Exercises

Try these on your own:

1. Find all people with `youtube_total` greater than 5
2. Select people who have BOTH `tiktok` > 0 AND `pinterest` > 0
3. Create a subset with only people who didn't purchase (`purchase == 0`)
4. Set `person_id` as the index on both dataframes
5. Find females with more than 1 child (hint: merge/join datasets or work separately)
6. Select the last 10 rows using `.iloc[]`
7. Find people with `tv_total` between 20 and 50 (use `&` with two conditions)
8. Count how many people have zero online exposure (`online_total == 0`)

### Bonus Challenges

9. Create a new column `high_tv` that's True when `tv_total > 50`, then filter on it
10. Combine the media and demo dataframes on `person_id` and filter by both media exposure and demographics
11. Find the top 10 people by `tv_total` (hint: use `.nlargest()`)

## Loading/Saving Data Between Notebooks

### Load Cleaned Data from Previous Notebook

If you cleaned data in notebook 01, you could load it here:

```python
# Uncomment to load cleaned data from previous notebook
# media_df = pd.read_csv('../outputs/media_cleaned.csv')
# demo_df = pd.read_csv('../outputs/demo_cleaned.csv')
```

### Save Your Work for Next Notebook

Save your filtered/selected data for use in later notebooks:

```python
# Uncomment to save your selections
# high_exposure.to_csv('../outputs/high_exposure.csv', index=False)
# purchasers.to_csv('../outputs/purchasers.csv', index=False)
```

**Note**: Make sure the `../outputs/` directory exists first!

In [ ]:
# Setup: imports and display options
import pandas as pd
import numpy as np

pd.options.display.max_columns = 50
pd.options.display.width = 120
pd.options.display.max_rows = 20

print("Setup complete!")
print(f"pandas version: {pd.__version__}")

## 1. Load Fresh Data

We'll start by loading the datasets fresh from the `../data/` folder:

**Note**: In a real workflow, you might load cleaned data from a previous notebook. See the commented section at the end for how to do that.